In [1]:
pip install selenium

In [4]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [18]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

# 使用 WebDriverManager 自動安裝 ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)  # 使用隱式等待

# 統一處理點擊函數
def safe_click(xpath, wait, retries=3):
    for attempt in range(retries):
        try:
            time.sleep(0.3)
            element = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
            driver.execute_script("arguments[0].scrollIntoView(true);", element)
            element.click()
            return
        except (StaleElementReferenceException, TimeoutException):
            print(f"元素無效或無法點擊，重試中（第 {attempt + 1} 次）...")
            if attempt == retries - 1:
                raise
        except ElementClickInterceptedException:
            print(f"元素被其他元素擋住，嘗試使用 JavaScript 點擊。")
            driver.execute_script("arguments[0].click();", element)
            return
    print(f"元素 {xpath} 點擊失敗，已嘗試多次。")


def add_meal_to_cart(meal_name, wait):
    try:
        meal_xpath = f"//h4[contains(text(), '{meal_name}')]"
        meal_element = wait.until(EC.presence_of_element_located((By.XPATH, meal_xpath)))
        try:
            sold_out_label = meal_element.find_element(By.XPATH, ".//span[contains(text(), '售完')]")
            if sold_out_label:
                print(f"{meal_name} 已售完，無法加入購物車。")
                return
        except NoSuchElementException: 
            print(f"{meal_name} 有庫存，繼續加入購物車過程。")
            safe_click(meal_xpath, wait)
            join_cart_button_xpath = "//button[contains(@class, 'join-cart-button')]"
            safe_click(join_cart_button_xpath, wait)
            print(f"{meal_name} 加入購物車成功")
    except (ElementClickInterceptedException, TimeoutException, StaleElementReferenceException):
        print(f"無法加入 {meal_name}，可能是按鈕被擋住或過期。")

def close_announcement(wait):
    try:
        announcement_button_xpath = '//*[@id="__nuxt"]/div/div/div[2]/div/section[1]/div[1]/div/div/header/button'
        safe_click(announcement_button_xpath, wait)
        print("找到公告並且關閉。")
    except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
        print("找不到公告按鈕或按鈕過期。")

def change_time(wait):
    try:
        open_time_xpath = '//*[@id="__nuxt"]/div/div/div[2]/div/div/section/div/div[1]'
        safe_click(open_time_xpath, wait)
        print("打開取餐資訊。")

        change_delivery_xpath = '/html/body/div[1]/div/div/div[2]/div/div[1]/section/div[2]/div/div/div[1]/div/section/div[2]/div/div/label[2]'
        safe_click(change_delivery_xpath, wait)
        print("更改為外送。")

        address_field = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//input[@class='el-input__inner' and @placeholder='輸入外送地址']")))
        address_field.send_keys("106台北市大安區大安路一段84號")

        save_button_xpath = '//*[@id="__nuxt"]/div/div/div[2]/div/div/section/div[2]/div/div/div[2]/button'
        safe_click(save_button_xpath, wait)
        print("儲存設定。")
    except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
        print("找不到取餐資訊。")

def pay_page(wait):
    try:
        open_cart_xpath = '//*[@id="__nuxt"]/div/div/div[2]/div/div[2]/div/div/section/div/button'
        safe_click(open_cart_xpath, wait)
        print("打開購物車")

        go_to_pay_page_xpath = '//*[@id="__nuxt"]/div/div/div[2]/div/div[3]/div/div/div[2]/div'
        safe_click(go_to_pay_page_xpath, wait)
        print("進入結帳頁")
    except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
        print("找不到按鈕或按鈕過期。")

def pay_money(wait):
    try:
        coupon_xpath = '//*[@id="__nuxt"]/div/div/div[2]/section/section/section[3]/section[1]/section/div/section[2]/section[2]/section[1]/section/section[1]/section/div/div[2]'
        try:
            safe_click(coupon_xpath, wait)
            print("刪除折價券")
        except NoSuchElementException:
            print("找不到折價券，可能已經被刪除或不存在。")
        except TimeoutException:
            print("刪除折價券超時，可能按鈕無法點擊或超出等待範圍。")
        except StaleElementReferenceException:
            print("元素引用過時，重試...")

        next_page1_xpath = '//*[@id="__nuxt"]/div/div/div[2]/section/div/div/section/div/button[2]'
        safe_click(next_page1_xpath, wait)
        print("進入付款資訊")

        payment_method_xpath = '//*[@id="payment-info-anchor"]/section/section[3]/section/div/button[1]'
        safe_click(payment_method_xpath, wait)
        print("選擇取餐付款")

        next_page2_xpath = '//*[@id="__nuxt"]/div/div/div[2]/section/div/div/section/div/button[2]'
        safe_click(next_page2_xpath, wait)
        print("進入確認結帳")

        send_order_xpath = '//*[@id="__nuxt"]/div/div/div[2]/section/div/div/section/div/button[2]'
        safe_click(send_order_xpath, wait)
        print("送出訂單")
    except NoSuchElementException:
        print("未進入結帳頁面。")

# 主程式流程
try:
    driver.get("https://dev.imenu.com.tw/damai/branches")
    driver.maximize_window()
    wait = WebDriverWait(driver, 10)

    # 點擊登入按鈕
    try:
        login_button_xpath = '//*[@id="memberButton"]/div[3]/button'
        safe_click(login_button_xpath, wait)

        account_field = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='帳號']")))
        password_field = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='密碼']")))
        account_field.send_keys("0989650932")
        password_field.send_keys("Brain3303")

        submit_button_xpath = '//*[@id="__nuxt"]/div/div/div[2]/form/div/button'
        safe_click(submit_button_xpath, wait)
        print("登入成功！")

        # 點擊指定分店
        try:
            branch_xpath = "//div[@class='branchItem' and .//h5[text()='大麥八德UAT測試測試']]"
            safe_click(branch_xpath, wait)
            print("成功點擊大麥八德UAT測試測試分店！")
        except (NoSuchElementException, TimeoutException):
            print("找不到大麥八德UAT測試測試分店或分店不可點擊。")

        close_announcement(wait)
        change_time(wait)

        # 使用通用函數加入不同的餐點到購物車
        add_meal_to_cart('烤肉吐司', wait)
        add_meal_to_cart('火腿吐司', wait)

        pay_page(wait)
        pay_money(wait)

    except (NoSuchElementException, TimeoutException):
        print("找不到登入按鈕或按鈕不可點擊。")

    # 等待用戶按鍵以保持瀏覽器開啟
    input("按任意鍵結束程式，並關閉瀏覽器。")

finally:
    driver.quit()

登入成功！
元素被其他元素擋住，嘗試使用 JavaScript 點擊。
成功點擊大麥八德UAT測試測試分店！
元素無效或無法點擊，重試中（第 1 次）...
元素無效或無法點擊，重試中（第 2 次）...
元素無效或無法點擊，重試中（第 3 次）...
找不到公告按鈕或按鈕過期。
元素被其他元素擋住，嘗試使用 JavaScript 點擊。
打開取餐資訊。
更改為外送。
儲存設定。
烤肉吐司 有庫存，繼續加入購物車過程。
元素被其他元素擋住，嘗試使用 JavaScript 點擊。
烤肉吐司 加入購物車成功
火腿吐司 有庫存，繼續加入購物車過程。
元素被其他元素擋住，嘗試使用 JavaScript 點擊。
火腿吐司 加入購物車成功
打開購物車
進入結帳頁
元素被其他元素擋住，嘗試使用 JavaScript 點擊。
刪除折價券
進入付款資訊
元素被其他元素擋住，嘗試使用 JavaScript 點擊。
選擇取餐付款
進入確認結帳
送出訂單


按任意鍵結束程式，並關閉瀏覽器。 a
